Run, develop and demonstrate a pipeline for searching for potential lensed supernovae.

Steps:
* Create an alert query token and start iterating through those.
* Filter these based on the minimal criteria of potential detections.
* Obtain redshift information from number of sources: masterlens, NED, SDSS photo-z, LS photo-z
* Fit SALT2 model on best redshift estimate, return estimated absolute peak magnitude
* Plot lightcurve fit + links to slack

The code assumes that the Ampel libraries are installed. Three steps are needed for this:
* Start a local MongoDB instance (with default connections).
* Download AMPEL v 0.8 core, ZTF and ampel-hu-astro libraries.
* Create a default ampel configuration file through running 'ampel-config build > ampel_conf.yaml'.

In [ ]:
import sys, os
import requests

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
header = {"Authorization": "bearer "+vault.get_named_secret("ztf/archive/token").value}
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
# at six month when 2458849.50000 at one year 2459031.50000 (from july 1st 2019)
query = {
  "jd": {
    "$gt": 2458900.,
    "$lt": 2458931.5,
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "magpsf": {
      "$gt": 18
    },
    "ndethist": {
      "$gt": 6,
      "$lte": 30
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
if not response.ok:
    print( 'Query creation failed' )
    print(response)

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel82/ampel_conf_28b5f6.yaml'

In [ ]:
# Select date of query with format e.g. '20210610'
channel = 'GeneralLens'
ITER_MAX = 1000000

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'LensQuery',
    purge_db = False,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# For accessing the datalab portal to LS photo-z. Replace the username with what matches with the password 
# entered above
dl_config = {
    'datalab_user' : 'datalab_user_id',
    'datalab_pwd' : 'datalab_user_pwd',
}

In [ ]:
# These are configuration parameters for the initial alert filter step.
# Note that this filter configuration assumes that a first filter based on nbr of points were made in the alert query
# The T0 stage will not be exposed to the full candidate history. 
filter_config = {
    'min_ndet': 1,
    'min_tspan': 0,
    'max_tspan': 50,
    'min_archive_tspan': 6,
    'max_archive_tspan': 50,    
    'min_rb': 0.3,
    'min_drb': 0.995,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'SDSS_spec' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['z', 'bptclass', 'subclass'],
            'all': False,
        },
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'GLADEv23' : {
            'use' : 'extcats',
            'rs_arcsec' : 10,
            'keys_to_append' : ['z', 'dist', 'dist_err', 'flag1', 'flag2', 'flag3'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },

    }
}

In [ ]:
# New, new, also with all categories?
ampelz_conf = {
    "max_redshift_category" : 7,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
        {
            "unit": "T2LSPhotoZTap",
            "config": dl_config,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },

    ]
}

In [ ]:
sncosmo_conf = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "unc" : 3,
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_%s.svg",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "title": {
            "format_str": "%s %s %s",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "fig_text": {
            "format_str": "%s %s \nz-source %s \nchisq %.2f ndof %s",
            "arg_keys": ["stock", "model", "redshift_kind", "chisq", "ndof"]
        },
        "width": 10,
        "height": 6,
        "id_mapper": "ZTFIdMapper",
        "disk_save": "/home/jnordin/tmp/ztfsalt",
    }
}

In [ ]:
# We now add a second sncosmo configuration, with scaled z
sncosmo_conf_zscale = {
    "sncosmo_model_name" : "salt2",
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "unc" : 3,
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    "plot_props": {
        "tags": ["SALT", "SNCOSMO"],
        "file_name": {
            "format_str": "%s_%s_%s.svg",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "title": {
            "format_str": "%s %s %s",
            "arg_keys": ["stock", "model", "redshift_kind"]
        },
        "fig_text": {
            "format_str": "%s %s \nz-source %s \nchisq %.2f ndof %s",
            "arg_keys": ["stock", "model", "redshift_kind", "chisq", "ndof"]
        },
        "width": 10,
        "height": 6,
        "id_mapper": "ZTFIdMapper",
        "disk_save": "/home/jnordin/tmp/ztfsalt2z",
    }
    "scale_z": 2.,
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "SimpleDecentFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2MatchBTS",
                                },
                                {
                                    "unit": "T2GetLensSNParameters",
                                    "config": sncosmo_conf,
                                },                            
                                {
                                    "unit": "T2GetLensSNParameters",
                                    "config": sncosmo_conf_zscale,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                            {
                                'unit': 'T2LSPhotoZTap',
                                'config': dl_config,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                                'Ampel distance' :['ampel_dist'],
                                            },
                                            'T2GetLensSNParameters' : {
                                                '(g-r) peak' : ['fit_metrics', 'g_r_colour_peak'],
                                                '(r-i) peak' : ['fit_metrics', 'r_i_colour_peak'],
                                                '(g-i) peak' : ['fit_metrics', 'g_i_colour_peak'],
                                                '(g-r) plus7' : ['fit_metrics', 'g_r_colour_plus7'],
                                                '(r-i) plus7' : ['fit_metrics', 'r_i_colour_plus7'],
                                                '(g-i) plus7' : ['fit_metrics', 'g_i_colour_plus7'],
                                                '(g-r) minus7' : ['fit_metrics', 'g_r_colour_minus7'],
                                                '(r-i) minus7' : ['fit_metrics', 'r_i_colour_minus7'],
                                                '(g-i) minus7' : ['fit_metrics', 'g_i_colour_minus7'],
                                                '(g-r) peak err' : ['fit_metrics', 'g_r_colour_peak_err'],
                                                '(r-i) peak err' : ['fit_metrics', 'r_i_colour_peak_err'],
                                                '(g-i) peak err' : ['fit_metrics', 'g_i_colour_peak_err'],
                                                '(g-r) plus7 err' : ['fit_metrics', 'g_r_colour_plus7_err'],
                                                '(r-i) plus7 err' : ['fit_metrics', 'r_i_colour_plus7_err'],
                                                '(g-i) plus7 err' : ['fit_metrics', 'g_i_colour_plus7_err'],
                                                '(g-r) minus7 err' : ['fit_metrics', 'g_r_colour_minus7_err'],
                                                '(r-i) minus7 err' : ['fit_metrics', 'r_i_colour_minus7_err'],
                                                '(g-i) minus7 err' : ['fit_metrics', 'g_i_colour_plus7_err'],
                                                'Peak g-band obs mag' : ['fit_metrics', 'obsmag_ztfg_peak'],
                                                'Peak r-band obs mag' : ['fit_metrics', 'obsmag_ztfr_peak'],
                                                'Peak i-band obs mag' : ['fit_metrics', 'obsmag_ztfi_peak'],
                                                'Plus7 g-band obs mag' : ['fit_metrics', 'obsmag_ztfg_plus7'],
                                                'Plus7 r-band obs mag' : ['fit_metrics', 'obsmag_ztfr_plus7'],
                                                'Plus7 i-band obs mag' : ['fit_metrics', 'obsmag_ztfi_plus7'],
                                                'Minus7 g-band obs mag' : ['fit_metrics', 'obsmag_ztfg_minus7'],
                                                'Minus7 r-band obs mag' : ['fit_metrics', 'obsmag_ztfr_minus7'],
                                                'Minus7 i-band obs mag' : ['fit_metrics', 'obsmag_ztfi_minus7'],
                                                'Peak B abs mag' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                            'T2MatchBTS' : {
                                                'Type' : ['bts_type'],
                                                'IAU ID': ['bts_IAUID'],
                                            },
                                            
                                        },
                                        'transient_table_schema' : {
                                            
                                            'T2LSPhotoZTap' : {
                                              'LSPhotoZ photo-z' : ['T2LSPhotoZTap', 'z_phot_mean'],
                                              'LSPhotoZ spec-z' : ['T2LSPhotoZTap', 'z_spec'],
                                              'LSPhotoZ dist2transient' : ['T2LSPhotoZTap', 'dist2transient'],
                                              'LSPhotoZ r' : ['T2LSPhotoZTap', 'dered_mag_r'],
                                              'LSPhotoZ z' : ['T2LSPhotoZTap', 'dered_mag_z'],
                                              'LSPhotoZ g' : ['T2LSPhotoZTap', 'dered_mag_g'],
                                              'LSPhotoZ w1' : ['T2LSPhotoZTap', 'dered_mag_w1'],
                                              'LSPhotoZ w2' : ['T2LSPhotoZTap', 'dered_mag_w2'],
                                              'LSPhotoZ w3' : ['T2LSPhotoZTap', 'dered_mag_w3'],
                                              'LSPhotoZ w4' : ['T2LSPhotoZTap', 'dered_mag_w4'],
                                            },
                                            
                                            'T2CatalogMatch': {
                                                'SDSS spec z' : ['SDSS_spec','z'],
                                                "NED z" : ['NEDz','z'],
                                                "NED dist" : ['NEDz','dist2transient'],
                                                "Glade z" : ['GLADEv23','z'],
                                                "LS_Zou Photo-z": ['LSPhotoZZou','photoz'],
                                                "LS_Zou dist": ['LSPhotoZZou','dist2transient'],
                                                "LS_Zou logMassBest": ['LSPhotoZZou','logMassBest'],
                                                "LS_Zou logMassInf": ['LSPhotoZZou','logMassInf'],
                                                "LS_Zou logMassSup": ['LSPhotoZZou','logMassSup'],
                                                "WiseCos Photo-z": ['wiseScosPhotoz','zPhoto_Corr'],
                                                "WiseCos dist": ['wiseScosPhotoz','dist2transient'],
                                            },
                                            
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : "/home/jnordin/tmp"
                                    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()

In [ ]:
# Trying the plot

In [ ]:
lcplot_config = {
    'lc_model' : 'salt2',
    'param' : ['x1','c'],
    'param_bounds': {'x1':[-10,10], 'c':[-1,3] },
    'bts_classes' : {
        'SN Ia' : ['SN Ia', 'SN Ia-91T'],
        'SN II' : ['SN II', 'SN IIP', 'SN IIb', 'SN IIb', 'SN IIn'],
        'SN Ibc' : ['SN Ic-BL', 'SN Ib/c', 'SN Ic', 'SN Ib', 'SN Ibn'],
        'SLSNL' : ['SLSN-I', 'SLSN-II'],
        'TDE' : ['TDE'],
    },
    'z_source' : 'AMPELz',
    'plot_dir': '/home/jnordin/tmp',
    'name_filter': {'ZTF name': 'ZTF', 'TNS ID': 'TNS'},
    'save_table': True,
    'unit_name': 'T2GetLensSNParameters',
    'include_absmag': True,
}


In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "PlotLightcurveSample",
                                "config": lcplot_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()

## Parse some output files

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_z = pd.read_csv('/home/jnordin/tmp/LensedTable_RealZ.csv')

In [ ]:
df_2z = pd.read_csv('/home/jnordin/tmp/LensedTable_DoubleZ.csv')

In [ ]:
df = pd.merge(df_z, df_2z, on="stock", suffixes=("_z", "_2z"))

In [ ]:
plt.plot(df['SALT2 Color_z'], df['SALT2 Color_2z'], 'o')
plt.axis([-0.5,1,-0.5,1])

In [ ]:
plt.plot(df['SALT2 X1_z'], df['SALT2 X1_2z'], 'o')
plt.axis([-5,5,-5,5])

In [ ]:
good_in_z = ( (df['SALT2 X1_z']>-4) & (df['SALT2 X1_z']<4) & (df['SALT2 Color_z']>-0.2) & (df['SALT2 X1_z']<0.3) )

In [ ]:
good_in_2z = ( (df['SALT2 X1_2z']>-4) & (df['SALT2 X1_2z']<4) & (df['SALT2 Color_2z']>-0.2) & (df['SALT2 X1_2z']<0.3) )

In [ ]:
ok_fit = ( (df['Ampel z_z']>0.15) & (df['SALT2 X1 (err)_2z']<1) & (df['SALT2 Color (err)_2z']<1) )

In [ ]:
df_sub = df[(good_in_2z & ~good_in_z & ok_fit) ]

In [ ]:
df_sub